In [ ]:
!pip install transformer datasets torch pandas
!pip install tokenizers
!pip install underthesea

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from underthesea import word_tokenize

# Đọc file Excel chứa dữ liệu bệnh và triệu chứng
try:
    df = pd.read_excel("./data/DataBase.xlsx")
except:
    print("Thiếu file hoặc sai đường dẫn file")

# Hàm tiền xử lý text
def clean_text(text):
    """
    - Kiểm tra NaN, trả về chuỗi rỗng nếu NaN
    - Chuyển text về dạng string
    - Xóa ký tự \
    - Thay xuống dòng (\n, \r) bằng khoảng trắng
    - Xóa khoảng trắng thừa
    - Tách từ tiếng Việt bằng underthesea
    """
    if pd.isna(text):
        return ""
    
    text = str(text)
    text = re.sub(r'\\', '', text)                      # xóa ký tự \
    text = text.replace('\n', ' ').replace('\r', ' ')  # thay xuống dòng bằng space
    text = re.sub(' +', ' ', text)                      # loại khoảng trắng thừa

    # tách từ tiếng Việt
    text_segment = word_tokenize(text, format="text")
    return text_segment

# Kiểm tra cột "Triệu chứng" tồn tại
if "Triệu chứng" in df.columns:
    # Tạo cột content ghép tên bệnh + triệu chứng
    df['content'] = df["Tên bệnh"] + " có triệu chứng là " + df["Triệu chứng"]

    # Tiền xử lý từng dòng
    symptoms = df["content"].apply(clean_text)

    # Loại bỏ dòng rỗng
    symptoms = symptoms[symptoms != ""]

    # Lưu kết quả vào file txt, mỗi dòng là 1 văn bản
    with open("./data/symptoms_train.txt", "w", encoding='utf-8') as f:
        for line in symptoms:
            f.write(line + "\n")

    print(f"Đã lưu file train với kích thước {len(symptoms)}")

else:
    print("Không tìm thấy cột triệu chứng")


Đã lưu file train với kích thước 581


In [ ]:
from transformers import PhobertTokenizer, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset

# Load tokenizer PhoBERT
tokenizer = PhobertTokenizer.from_pretrained("vinai/phobert-base")

def tokenize_function(examples):
    """
    Tokenize các văn bản đầu vào:
      - examples["text"] là list các string
      - truncation=True để cắt dài quá max_length
      - max_length=256 token
    """
    return tokenizer(examples["text"], truncation=True, max_length=256)

# Load dataset từ file txt
raw_datasets = load_dataset(
    'text',
    data_files={'data': './data/symptoms_train.txt'}  # file txt train
)

# Chia train và test (90% train và 10% test)
split_datasets = raw_datasets["data"].train_test_split(
    test_size=0.1,
    seed=42
)

# Tokenize và loại bỏ cột gốc "text"
train_dataset = split_datasets["train"].map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

test_dataset = split_datasets["test"].map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

print(f"Số lượng mẫu train: {len(train_dataset)}")
print(f"Số lượng mẫu test: {len(test_dataset)}")

# Data collator cho Language Modeling (MLM)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,                 # Masked Language Modeling
    mlm_probability=0.15      # Xác suất mask 15% token (80% token được mask, 10% giữ nguyên, 10% ngẫu nhiên tránh mô hình quá tập trung vào dự đoán token mask)
)


In [ ]:
!pip install wandb

In [ ]:
from transformers import RobertaForMaskedLM, Trainer, TrainingArguments

# Load model PhoBERT chuẩn cho Masked Language Modeling
# Đây là model gốc, sẽ fine-tune trên dữ liệu triệu chứng y tế
model = RobertaForMaskedLM.from_pretrained("vinai/phobert-base")

# Cấu hình training
training_args = TrainingArguments(
    output_dir="./model/phobert-medical",   # thư mục lưu checkpoint
    overwrite_output_dir=True,              # ghi đè nếu đã có model cũ
    num_train_epochs=3,                     # số epoch
    per_device_train_batch_size=8,          # batch size cho mỗi GPU/CPU
    save_steps=500,                         # lưu checkpoint mỗi 500 step
    save_total_limit=2,                     # giữ tối đa 2 checkpoint
    learning_rate=2e-5,                     # learning rate
    weight_decay=0.01,                      # weight decay cho optimizer
    prediction_loss_only=True,              # chỉ trả loss khi eval
    report_to="wandb",                      # log tới Weights & Biases
    run_name="phobert-medical-final",       # tên experiment trên wandb
    logging_dir="./logs",                    # thư mục lưu log
    logging_steps=30,                        # log mỗi 30 step
    eval_strategy="steps",                   # eval theo step
    eval_steps=30                            # eval mỗi 30 step
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  # Data collator cho MLM
    train_dataset=train_dataset,  # dataset train đã tokenize
    eval_dataset=test_dataset,    # dataset test đã tokenize
    tokenizer=tokenizer           # tokenizer PhoBERT
)

# Bắt đầu huấn luyện
trainer.train()

# Lưu model và tokenizer sau fine-tune
trainer.save_model("./model/phobert-medical-final")  # lưu model
tokenizer.save_pretrained("./model/phobert-medical-final")  # lưu tokenizer

print("Huấn luyện hoàn tất và đã lưu model")


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipython-input-1550569193.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tamphu250305 (tamphu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
30,2.192400,1.975331
60,2.116700,2.030044
90,2.013600,1.919804
120,1.886300,1.796645
150,1.884400,1.785400
180,1.860500,1.787920


Huấn luyện hoàn tất và đã lưu model


In [ ]:
from transformers import pipeline

# 1) Tạo pipeline cho Masked Language Modeling
# Sử dụng model PhoBERT đã fine-tune
fill_mask = pipeline(
    "fill-mask",                                # task MLM
    model="./model/phobert-medical-final",     # đường dẫn model fine-tuned
    tokenizer="./model/phobert-medical-final"  # tokenizer tương ứng
)

# Ví dụ text có token mask
example_text = "Bệnh nhân có triệu chứng sốt cao và <mask> đầu."

# Dự đoán các token bị mask
predict = fill_mask(example_text)

# In kết quả dự đoánư
for pred in predict:
    print(f"{pred['sequence']} (Score: {pred['score']:.4f})")


Device set to use cuda:0


Bệnh nhân có triệu chứng sốt cao và viêm đầu. (Score: 0.2087)
Bệnh nhân có triệu chứng sốt cao và rối_loạn đầu. (Score: 0.0728)
Bệnh nhân có triệu chứng sốt cao và đau đầu. (Score: 0.0639)
Bệnh nhân có triệu chứng sốt cao và nấm đầu. (Score: 0.0504)
Bệnh nhân có triệu chứng sốt cao và hội_chứng đầu. (Score: 0.0414)
